In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy
import datetime

In [3]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, TensorDataset

In [4]:
cwd = os.getcwd()
print(cwd)
# cwd_parent = os.path.abspath(os.path.join(cwd, os.pardir))
cwd_parent = os.path.abspath(os.path.join(cwd, '../../'))
print(cwd_parent)

sys.path.append(cwd_parent)

/cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/notebooks/Baselines
/cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git


In [5]:
import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.chemfeatures import *
from deepadr.train_functions_DeepDDS import *
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN

In [6]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [7]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

In [8]:
report_available_cuda_devices()

number of GPUs available: 5
cuda:0, name:NVIDIA GeForce GTX 1080 Ti
total memory available: 10.91656494140625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:1, name:NVIDIA GeForce GTX 1080 Ti
total memory available: 10.91656494140625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:2, name:NVIDIA GeForce GTX 1080 Ti
total memory available: 10.91656494140625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:3, name:NVIDIA GeForce GTX 1080 Ti
total memory available: 10.91656494140625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:4, 

In [9]:
n_gpu = torch.cuda.device_count()
n_gpu

5

In [10]:
device_cpu = get_device(to_gpu=False)
# device_gpu = get_device(True, index=0)

In [11]:
print("torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print(sys.version)

torch: 1.11.0+cu115
CUDA: 11.5
3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]


### Preparing dataset 

In [12]:
# options: 
# 'total_thresh' + 4,3,2
# 'loewe_thresh', 'hsa_thresh', 'bliss_thresh', 'zip_thresh' + 1

score = 'total_thresh'
score_val = 4

In [13]:
DSdataset_name = f'DrugComb_{score}_{score_val}'

data_fname = 'data_v1' # v2 for baseline models

In [14]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# # ReaderWriter.dump_data(dpartitions, os.path.join(targetdata_dir, 'data_partitions.pkl'))
print(targetdata_dir)

path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
/cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/data/processed/DrugComb_total_thresh_4/data_v1


In [15]:
%%time

# Make sure to first run the "DDoS_Dataset_Generation" notebook first

dataset = MoleculeDataset(root=targetdata_dir)

CPU times: user 19 ms, sys: 351 ms, total: 370 ms
Wall time: 368 ms


In [16]:
dataset.data.y

tensor([0, 0, 0,  ..., 0, 1, 1], dtype=torch.int32)

In [17]:
fold_partitions = get_stratified_partitions(dataset.data.y,
                                            num_folds=5,
                                            valid_set_portion=0.1,
                                            random_state=42)

fold_num: 0
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 1
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 2
train data
class: 0 norm count: 0.5764356969533567
class: 1 norm count: 0.4235643030466433
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5763929334109882
class: 1 norm count: 0.4236070665890118

-------------------------
fold_num: 3
train data
class: 0 norm count: 0.576

In [18]:
print("Number of training graphs: "+ str(len(fold_partitions[0]['train'])))
print("Number of validation graphs: "+ str(len(fold_partitions[0]['validation'])))
print("Number of testing graphs: "+ str(len(fold_partitions[0]['test'])))

Number of training graphs: 18544
Number of validation graphs: 2061
Number of testing graphs: 5152


In [19]:
# print(dataset.num_classes)

In [20]:
# training parameters

tp = {
    "batch_size" : 300,
    "num_epochs" : 100,
    
    "emb_dim" : 128,
    "gnn_type" : "gat",
    "num_layer" : 5,
    "graph_pooling" : "max", #attention
    
    "input_embed_dim" : None,
    "gene_embed_dim": 1,
    "num_attn_heads" : 10,
    "num_transformer_units" : 1,
    "p_dropout" : 0.2,
#     "nonlin_func" : nn.ReLU(),
    "mlp_embed_factor" : 2,
    "pooling_mode" : 'attn',
    "dist_opt" : 'cosine',

    "base_lr" : 3e-4, #3e-4
    "max_lr_mul": 10,
    "l2_reg" : 1e-7,
    "loss_w" : 1.,
    "margin_v" : 1.,

    "expression_dim" : 64,
    "expression_input_size" : 908,
    "exp_H1" : 2048,
    "exp_H2" : 512
}

In [21]:
def spawn_q_process(q_process):
    print(">>> spawning hyperparam search process")
    q_process.start()
    
def join_q_process(q_process):
    q_process.join()
    print("<<< joined hyperparam search process")
    
def create_q_process(queue, used_dataset, gpu_num, tphp, exp_dir, partition):
    return mp.Process(target=deepadr.train_functions_DeepDDS.run_exp_deepdds, args=(queue, used_dataset, gpu_num, tphp, exp_dir, partition))

In [22]:
import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)

queue = mp.Queue()
q_processes = []

time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

print("Start: " + time_stamp)

for q_i in range(min(n_gpu, len(fold_partitions))):
    partition = fold_partitions[q_i]
    exp_dir = create_directory(os.path.join(targetdata_dir_exp, "fold_"+str(q_i)+"_"+time_stamp))
    create_directory(os.path.join(exp_dir, "predictions"))
    
    q_process = create_q_process(queue, dataset, q_i, tp, exp_dir, partition)
    q_processes.append(q_process)
    spawn_q_process(q_process)

spawned_processes = n_gpu
    
for q_i in range(min(n_gpu, len(fold_partitions))):
    join_q_process(q_processes[q_i])
    released_gpu_num = queue.get()
    print("released_gpu_num:", released_gpu_num)

Start: 2023-03-08_14-32-40
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
>>> spawning hyperparam search process
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
>>> spawning hyperparam search process


Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

gpu: cuda:0
Dropout(p=0.2, inplace=False) Dropout(p=0.2, inplace=False)
=====Epoch 0
Training...
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
>>> spawning hyperparam search process


Iteration:   2%|▏         | 1/62 [00:02<02:57,  2.92s/it]

gpu: cuda:1
Dropout(p=0.2, inplace=False) Dropout(p=0.2, inplace=False)
=====Epoch 0
Training...
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
path_current_dir /cluster/work/medinfmk/lm1-homes/skyriakos-lm1-home/data_to_migrate/chemprop_run/git/deepadr
>>> spawning hyperparam search process


...

{'Train':  best_epoch_indx:4
 auc:0.777600421341206 
 apur:0.7389481822482461 
 f1:0.5874379383718181 
 precision:0.7709971038477451 
 recall:0.4744748567791216 
, 'Validation':  best_epoch_indx:4
 auc:0.7541383672540602 
 apur:0.7124934143844229 
 f1:0.5535585909417685 
 precision:0.7432432432432432 
 recall:0.44100801832760594 
, 'Test':  best_epoch_indx:4
 auc:0.7676131050491497 
 apur:0.7293010433920817 
 f1:0.5862552594670406 
 precision:0.7556037599421548 
 recall:0.47891842346471125 
}
Finished training!


Iteration:  43%|████▎     | 3/7 [00:00<00:00, 27.63it/s]

<<< joined hyperparam search process
released_gpu_num: 3


Iteration: 100%|██████████| 7/7 [00:00<00:00, 28.49it/s]


{'Train':  best_epoch_indx:4
 auc:0.7854323957558627 
 apur:0.7429424870999152 
 f1:0.6354412786657401 
 precision:0.6996174445294567 
 recall:0.5820496499045195 
, 'Validation':  best_epoch_indx:4
 auc:0.7768328570161329 
 apur:0.7321161486817652 
 f1:0.6265822784810126 
 precision:0.7001414427157001 
 recall:0.5670103092783505 
, 'Test':  best_epoch_indx:4
 auc:0.7863342069086026 
 apur:0.7452965506444699 
 f1:0.6297700277988376 
 precision:0.7019718309859155 
 recall:0.5710357470210816 
}
Finished training!
<<< joined hyperparam search process
released_gpu_num: 4


In [23]:
print("End: " + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))


End: 2023-03-08_14-33-47
